<a href="https://colab.research.google.com/github/sightes/RSS_Explorer/blob/main/RSS_Scrapping_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict
!pip install html2text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
import requests
import xmltodict
import json
import numpy as np
import html2text
import lxml.html
import pandas as pd 

class RSS_topics:
    def __init__(self):
      self.url='www.default.com'

    def get_parse(self,noticia,text):
      h = html2text.HTML2Text()
      h.ignore_links = True
      h.ignore_tables=True
      h.ignore_images=True
      h.ignore_emphasis=True
      h.body_width=0
      return(h.handle(noticia[text]).replace('\n',' ').replace("\ ",' '))

    def safe_execute(self,default, exception, function, *args):
        try:
            return function(*args)
        except:
            return default

    def request_xml(self,url):
      response= requests.get(url)
      data= xmltodict.parse(response.content)
      return(json.dumps(data))

    def parse_items(self,data):  
      parsed_data=pd.DataFrame([])
      n_noticias=len(json.loads(data)['rss']['channel']['item'])
      noticias=json.loads(data)['rss']['channel']['item']
      parsed_data['sitio']=[json.loads(data)['rss']['channel']['title'] for c_noticia in noticias]
      parsed_data['descripcion_sitio']=[json.loads(data)['rss']['channel']['description'] for c_noticia in noticias]
      parsed_data['titulares']=[self.safe_execute('',1,self.get_parse,c_noticia,'title') for c_noticia in noticias]
      parsed_data['description']=[self.safe_execute('',1,self.get_parse,c_noticia,'description') for c_noticia in noticias]
      parsed_data['contenido']=[self.safe_execute('',1,self.get_parse,c_noticia,'content:encoded') for c_noticia in noticias]
      parsed_data['categorias']=[self.safe_execute('',1,self.get_parse,c_noticia,'category') for c_noticia in noticias]
      return(parsed_data)


    def get_fuentes_feedspot(self):
      r= requests.get('https://blog.feedspot.com/mexican_news_rss_feeds/')
      root = lxml.html.fromstring(r.text)
      paginas=root.xpath("////*/a[@class = 'ext']/@href")
      selected=np.where([i!='' for i in paginas])[0].tolist()
      self.feedspot=[paginas[i] for i in range(0,len(paginas)) if i in selected]
      return(self)

    def get_fuentes_investing(self):
      r= requests.get('https://mx.investing.com/webmaster-tools/rss')
      root = lxml.html.fromstring(r.text)
      paginas=root.xpath("////*/a[@class= 'rssDescription inlineblock']/@href")
      selected=np.where([i!='' for i in paginas])[0].tolist()
      self.investing=[paginas[i] for i in range(0,len(paginas)) if i in selected]
      return(self)


    def get_fuentes_elheraldo(self):
      r= requests.get('https://heraldodemexico.com.mx/rss')
      root = lxml.html.fromstring(r.text)
      paginas=root.xpath("////*/a[@class = 'link-lineal']/@href")
      selected=np.where([i!='' for i in paginas])[0].tolist()
      self.elheraldo=[paginas[i] for i in range(0,len(paginas)) if i in selected]
      return(self)

    
    def get_fuentes_custom(self):
      self.custom=['https://www.eleconomista.com.mx/rss/ultimas-noticias']
      return(self)

    def parse_iter_rss(self):
      try:
        fuentes=self.feedspot+self.elheraldo+self.custom+self.investing
      except:
        fuentes=self.get_fuentes_feedspot()+self.get_fuentes_elheraldo()+self.get_fuentes_custom()+self.get_fuentes_investing()
      consolidado=pd.DataFrame([],columns=['sitio','descripcion_sitio','titulares','description','contenido','categorias'])
      for i in fuentes:
        try:
          c_fuente=self.parse_items(self.request_xml(i))
          consolidado=consolidado.append(c_fuente)
          print('Ok! '+ i)
        except:
          print('no hubo conexion con '+ i)
      return(consolidado)
      

In [73]:
rss=RSS_topics()
rss=rss.get_fuentes_feedspot()
rss=rss.get_fuentes_elheraldo()
rss=rss.get_fuentes_investing()
rss=rss.get_fuentes_custom()

In [74]:
rss.parse_iter_rss()

Ok! https://www.debate.com.mx/rss/feed.xml
no hubo conexion con https://www.excelsior.com.mx/rss.xml
no hubo conexion con https://www.reforma.com/rss/portada.xml
Ok! https://vanguardia.com.mx/rss.xml
Ok! https://www.elsiglodetorreon.com.mx/index.xml
Ok! https://mexiconewsdaily.com/feed/
Ok! https://www.razon.com.mx/feed/
Ok! https://www.jornada.com.mx/feeds/rss
Ok! https://diario.mx/jrz/media/sitemaps/rss.xml
Ok! https://www.elfinanciero.com.mx/rss
Ok! https://feeds.sdpnoticias.com/portal/all
no hubo conexion con https://www.sinembargo.mx/feed
Ok! https://www.publimetro.com.mx/mx/rss/global.xml
no hubo conexion con https://www.elnorte.com/rss/portada.xml
Ok! https://eldeforma.com/feed/
Ok! https://www.informador.mx/rss/mexico.xml
Ok! https://www.24-horas.mx/feed/
Ok! http://imparcialoaxaca.mx/feed/
no hubo conexion con https://www.mural.com.mx/rss/portada.xml
no hubo conexion con https://diariobasta.com/feed/
Ok! https://www.theyucatantimes.com/feed/
Ok! https://8columnas.com.mx/feed/


,sitio,descripcion_sitio,titulares,description,contenido,categorias
0,DEBATE,Últimas Noticias de México y el Mundo,"Champotón, playa con mucha tradición de Campec...",Champoton es un municipio de Campeche con hist...,"Champotón, Campeche.- Champotón es un municipi...",Viajes
1,DEBATE,Últimas Noticias de México y el Mundo,Yucatán: Detienen a secuestrador de empresario...,Este sabado 21 de enero en Yucatan se informo ...,"Mérida, Yucatán.- Este sábado 21 de enero en Y...",Estados
2,DEBATE,Últimas Noticias de México y el Mundo,"Santos: ""Obtuvimos recompensa en una cancha co...",El entrenador lagunero se queda con la actitud...,"Mazatlán, Sinaloa.- Santos Laguna sufrió en la...",Deporte
3,DEBATE,Últimas Noticias de México y el Mundo,Cuándo será el concierto de RBD en Guadalajara...,La preventa de los boletos para el concierto d...,"Guadalajara, Jalisco.- La banda de pop que mar...",Guadalajara
4,DEBATE,Últimas Noticias de México y el Mundo,Denuncian a clínica del IMSS en Durango por qu...,"La bebe nacio sin ninguna complicacion, fue qu...",Durango.- Una recién nacida se encuentra en cu...,Policíaca
...,...,...,...,...,...,...
95,El Economista,Diario de Economía.,La geopolítica del caos: América Latina,,,
96,El Economista,Diario de Economía.,Colombia extradita a EU a hermano de senadora ...,El gobierno de Colombia extradito ayer a Estad...,,
97,El Economista,Diario de Economía.,"Municipios, sin interés en planes de desarrollo",Del total de ayuntamientos en el territorio na...,,
98,El Economista,Diario de Economía.,Querétaro retoma la atracción de IED estadouni...,"Durante enero-septiembre del 2022, Queretaro c...",,


In [77]:
self.investing[0]

NameError: ignored

In [75]:
r= requests.get('https://servicios.elpais.com/rss/')
root = lxml.html.fromstring(r.text)
paginas=root.xpath("////*/a/strong")
paginas      

[<Element strong at 0x7fdc21f37a90>,
 <Element strong at 0x7fdc21f6a770>,
 <Element strong at 0x7fdc21f6abd0>,
 <Element strong at 0x7fdc21f6a9a0>,
 <Element strong at 0x7fdc21f6aa40>,
 <Element strong at 0x7fdc21f6aa90>,
 <Element strong at 0x7fdc21f6ac20>,
 <Element strong at 0x7fdc21f6ad10>,
 <Element strong at 0x7fdc21f6a630>,
 <Element strong at 0x7fdc21f6a7c0>,
 <Element strong at 0x7fdc21f6ab80>,
 <Element strong at 0x7fdc21f6a860>,
 <Element strong at 0x7fdc21f6aae0>,
 <Element strong at 0x7fdc21f6ab30>,
 <Element strong at 0x7fdc21f6acc0>,
 <Element strong at 0x7fdc21f6a950>,
 <Element strong at 0x7fdc21f6a590>,
 <Element strong at 0x7fdc21f6a400>,
 <Element strong at 0x7fdc21f6aea0>,
 <Element strong at 0x7fdc21f6a900>,
 <Element strong at 0x7fdc21f6af40>,
 <Element strong at 0x7fdc21f6a720>,
 <Element strong at 0x7fdc21e4d630>,
 <Element strong at 0x7fdc21e4d9a0>,
 <Element strong at 0x7fdc21e4d810>,
 <Element strong at 0x7fdc21e4df90>,
 <Element strong at 0x7fdc21e4dcc0>,
 

In [71]:
rss.investing

['https://mx.investing.com/rss/market_overview_Technical.rss',
 'https://mx.investing.com/rss/market_overview_Fundamental.rss',
 'https://mx.investing.com/rss/market_overview_Opinion.rss',
 'https://mx.investing.com/rss/market_overview_investing_ideas.rss',
 'https://mx.investing.com/rss/forex_Technical.rss',
 'https://mx.investing.com/rss/forex_Fundamental.rss',
 'https://mx.investing.com/rss/forex_Opinion.rss',
 'https://mx.investing.com/rss/forex_Signals.rss',
 'https://mx.investing.com/rss/stock_Technical.rss',
 'https://mx.investing.com/rss/stock_Fundamental.rss',
 'https://mx.investing.com/rss/stock_Opinion.rss',
 'https://mx.investing.com/rss/stock_stock_picks.rss',
 'https://mx.investing.com/rss/stock_Stocks.rss',
 'https://mx.investing.com/rss/stock_Indices.rss',
 'https://mx.investing.com/rss/stock_Futures.rss',
 'https://mx.investing.com/rss/stock_Options.rss',
 'https://mx.investing.com/rss/commodities_Technical.rss',
 'https://mx.investing.com/rss/commodities_Fundamental.r